In [8]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
import black
import jupyter_black

jupyter_black.load(
    lab=False,
    line_length=79,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
    
)
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn.preprocessing as sp
import pickle
import matplotlib.pyplot as plt
import scipy
import os
import time
from datetime import date
# import dask.dataframe as dd
# import dask.config
# dask.config.set({"distributed.scheduler.allowed-failures": 10})
today = date.today()
import os
import sys
sys.path.insert(0, '/home/ubuntu/workspace_SingleCell/SingleCell_Morphological_Analysis/') 
from singlecell.read import read_single_cell_sql
from singlecell.preprocess import handle_nans, extract_cpfeature_names,find_highly_correlated_features
from singlecell.visualize import visualize_n_SingleCell, cluster
from singlecell.process import statistical_tests,precision_recall
from singlecell.preprocess.filter_out_edge_single_cells import edgeCellFilter
from singlecell.save.save_pandas_dfs import saveDF_to_CSV_GZ_no_timestamp
from singlecell.preprocess.control_for_cellcount import control_feature_y_for_variable_x
from singlecell.process.replicate_correlation import replicate_null_corr_coefs
from singlecell.process import normalize_funcs


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
########################## Project root directory and path to results ########################
mito_project_root_dir = "/home/ubuntu/bucket/projects/2016_08_01_RadialMitochondriaDistribution_donna/"
save_results_dir = mito_project_root_dir + "/workspace/results/jump_fq/"

In [9]:
import pandas as pd
from sqlalchemy import create_engine
from functools import reduce
import gc


def read_per_well_data(
    input_data_dir,
    annot,
    prof_workspace_folder_name="profiles",
    fformat=".parquet",
):
    batches = annot["Batch"].unique()

    df_agg_all_batches_ls = []
    for b in batches:
        print(b)
        #         if "Metadata_Source" in annot.columns:
        source_str = annot.loc[
            annot["Batch"] == b, "Metadata_Source"
        ].unique()[0]
        #             print(source_str)
        profile_path = (
            input_data_dir
            + source_str
            + "/workspace/"
            + prof_workspace_folder_name
            + "/"
        )
        #         else:
        #             profile_path = input_data_dir + "/workspace/profiles/"

        df_sag_ls = []
        plates_exist = os.listdir(profile_path + b)
        plates_meta = annot.loc[annot["Batch"] == b, "Metadata_Plate"].unique()
        plates = set(plates_meta) & set(plates_exist)
        for p in plates:
            print(p)

            fileName = profile_path + b + "/" + p + "/" + p + fformat
            #             print(fileName)
            if os.path.exists(fileName):
                if fformat == ".parquet":
                    sc_df = pd.read_parquet(fileName)
                elif fformat in [".csv", ".csv.gz"]:
                    sc_df = pd.read_csv(fileName)

                #         per_site_aggregate=sc_df.groupby(['Metadata_Well','Metadata_Site']).mean()[feature_list+['Count_Cells']].reset_index()
                sc_df["Metadata_Batch"] = b
                sc_df["Metadata_Plate"] = p
                df_sag_ls.append(sc_df)
                del sc_df
                gc.collect()
            else:
                print(fileName, " not exists")

        if df_sag_ls:
            df_sag = pd.concat(df_sag_ls, axis=0)
            df_agg_all_batches_ls.append(df_sag)

    df_agg_all_batches = pd.concat(
        df_agg_all_batches_ls, axis=0, ignore_index=True
    )
    return df_agg_all_batches

In [11]:
lincs_meta_cols = [
    "Metadata_broad_sample",
    "Metadata_dose_recode",
    "Metadata_pert_id",
    "Metadata_pert_mfc_id",
    "Metadata_InChIKey14",
    "Metadata_pert_type",
    "Metadata_moa",
    "Metadata_target",
    "Metadata_pert_id_dose",
    "Metadata_pert_name",
]

# lincs_meta_cols=['Metadata_broad_sample','Metadata_dose_recode','Metadata_pert_id','Metadata_pert_mfc_id',\
# 'Metadata_InChIKey14','Metadata_pert_type','Metadata_pert_id_dose']

cdrp_meta_cols = [
    "Metadata_broad_sample",
    "Metadata_mmoles_per_liter2",
    "Metadata_pert_id",
    "Metadata_Sample_Dose",
    "Metadata_moa",
]
jumporf_meta_cols = ["Symbol", "broad_sample"]
jumpcrispr_meta_cols = ["Metadata_NCBI_Gene_ID", "Metadata_Symbol"]
jumpcompound_meta_cols = ["Metadata_InChIKey", "Metadata_InChI"]
taorf_meta_cols = [
    "Metadata_gene_name",
    "Metadata_pert_name",
    "Metadata_broad_sample",
    "Metadata_moa",
]


# jump_orf_params={'profiles_path':"/home/ubuntu/jumpbucket/projects/2021_04_26_Production/workspace/backend/",\
#                  'meta_cols':jumporf_meta_cols,\
#                  'pert_col':'broad_sample',\
#                  'target_features_list':target_features_list_orf_cdrp
#                 }

jump_orf_params = {
    "profiles_path": "/home/ubuntu/gallery/cpg0016-jump/",
    "prof_workspace_folder_name": "profiles",
    "pformat": ".parquet",
    "meta_cols": jumporf_meta_cols,
    "pert_col": "Metadata_JCP2022",
    "untreated_key_val": ["Metadata_pert_type", "negcon"],
}

cdrp_params = {
    "profiles_path": "/home/ubuntu/gallery/cpg0012-wawer-bioactivecompoundprofiling/",
    "prof_workspace_folder_name": "profiles",
    "pformat": ".csv.gz",
    "meta_cols": cdrp_meta_cols,
    "pert_col": "Metadata_Sample_Dose",
    "untreated_key_val": ["Metadata_pert_type", "control"],
}


# /home/ubuntu/bucket/projects/2015_10_05_DrugRepurposing_AravindSubramanian_GolubLab_Broad/workspace/backend/
# https://cellpainting-gallery.s3.amazonaws.com/cpg0004-lincs/broad/workspace/backend/2016_04_01_a549_48hr_batch1/SQ00014812
lincs_g_params = {
    "profiles_path": "/home/ubuntu/gallery/cpg0004-lincs/",
    "prof_workspace_folder_name": "backend",
    "pformat": ".csv",
    "meta_cols": lincs_meta_cols,
    "pert_col": "Metadata_pert_id_dose",
    "untreated_key_val": ["Metadata_pert_type", "control"],
}

lincs_params = {
    "profiles_path": "/home/ubuntu/bucket/projects/2015_10_05_DrugRepurposing_AravindSubramanian_GolubLab_Broad/workspace/backend/",
    "prof_workspace_folder_name": "backend",
    "pformat": ".csv",
    "meta_cols": lincs_meta_cols,
    "pert_col": "Metadata_pert_id_dose",
    "untreated_key_val": ["Metadata_pert_type", "control"],
}

jump_crispr_params = {
    "profiles_path": "/home/ubuntu/gallery/cpg0016-jump/",
    "prof_workspace_folder_name": "profiles",
    "pformat": ".parquet",
    "meta_cols": jumpcrispr_meta_cols,
    "pert_col": "Metadata_JCP2022",
    "untreated_key_val": ["Metadata_Symbol", "non-targeting"],
}

jump_compound_params = {
    "profiles_path": "/home/ubuntu/gallery/cpg0016-jump/",
    "prof_workspace_folder_name": "profiles",
    "pformat": ".parquet",
    "meta_cols": jumpcompound_meta_cols,
    "pert_col": "Metadata_JCP2022",
    "untreated_key_val": ["Metadata_JCP2022", "JCP2022_999999"],
}

ta_orf_params = {
    "profiles_path": "/home/ubuntu/gallery/cpg0017-rohban-pathways/",
    "prof_workspace_folder_name": "profiles",
    "pformat": ".csv.gz",
    "meta_cols": taorf_meta_cols,
    "pert_col": "Metadata_broad_sample",
    "untreated_key_val": ["Metadata_pert_type", "Untreated"],
}

ds_info_dict = {
    "jump_orf": jump_orf_params,
    "CDRP": cdrp_params,
    "lincs": lincs_params,
    "lincs_g": lincs_g_params,
    "jump_crispr": jump_crispr_params,
    "jump_compound": jump_compound_params,
    "taorf": ta_orf_params,
}
# 'broad_sample', 'pert_type', 'control_type'

# results=annot[['Symbol','broad_sample', 'pert_type', 'control_type']].drop_duplicates().reset_index(drop=True)

# dataset='CDRP';dataset_meta_hue='Metadata_moa'
# dataset='lincs';dataset_meta_hue='Metadata_moa'
# dataset='jump_orf';dataset_meta_hue='Symbol'

In [190]:
dataset = "jump_crispr"
dataset = "jump_compound"
dataset = "jump_orf"
# dataset = "lincs"
# dataset="taorf"
dataset = "CDRP"

##################### Read preprocessed metadata
annot = pd.read_csv(
    mito_project_root_dir
    + "/workspace/metadata/preprocessed/annot_"
    + dataset
    + ".csv",
    dtype={"Metadata_Plate": str},
)
# annot["Metadata_pert_type"]
annot

/tmp/ipykernel_26964/163164690.py:9: DtypeWarning: Columns (17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  annot = pd.read_csv(


,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_ASSAY_WELL_ROLE,Metadata_broad_sample,Metadata_mmoles_per_liter,Metadata_solvent,Metadata_pert_id,...,Metadata_pert_type,Metadata_pert_iname,Metadata_pert_iname2,Metadata_moa,Metadata_target,Metadata_mmoles_per_liter2,Metadata_Sample_Dose,Batch,batch_plate,ctrl_well
0,24277,a01,24277,H-BIOA-004-3,a01,treated,BRD-K18250272-003-03-7,3.022516,DMSO,BRD-K18250272,...,trt,propoxycaine,propoxycaine,local anesthetic,NaN,6.05,BRD-K18250272-003-03-7_6.05,CDRP,CDRP-24277,False
1,24277,a02,24277,H-BIOA-004-3,a02,treated,BRD-K18316707-001-01-9,5.000000,DMSO,BRD-K18316707,...,trt,O-1918,BRD-K18316707,cannabinoid receptor antagonist,NaN,10.00,BRD-K18316707-001-01-9_10.0,CDRP,CDRP-24277,False
2,24277,a03,24277,H-BIOA-004-3,a03,treated,BRD-K18438502-001-02-6,5.000000,DMSO,BRD-K18438502,...,trt,NaN,NaN,NaN,NaN,10.00,BRD-K18438502-001-02-6_10.0,CDRP,CDRP-24277,False
3,24277,a04,24277,H-BIOA-004-3,a04,treated,BRD-K18550767-001-02-8,5.000000,DMSO,BRD-K18550767,...,trt,bergenin,bergenin,interleukin inhibitor,"IL1B, TNF",10.00,BRD-K18550767-001-02-8_10.0,CDRP,CDRP-24277,False
4,24277,a05,24277,H-BIOA-004-3,a05,treated,BRD-K18574842-323-03-3,2.195487,DMSO,BRD-K18574842,...,trt,nafcillin,nafcillin,bacterial cell wall synthesis inhibitor,"CYP1A2, CYP3A4, SLC22A6",4.39,BRD-K18574842-323-03-3_4.39,CDRP,CDRP-24277,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153381,26795,p20,26795,H-CBLD-004-4,p20,treated,BRD-K17405724-001-01-0,4.916061,DMSO,BRD-K17405724,...,trt,NaN,NaN,NaN,NaN,9.83,BRD-K17405724-001-01-0_9.83,CDRP,CDRP-26795,False
153382,26795,p21,26795,H-CBLD-004-4,p21,treated,BRD-K90170141-001-01-5,4.859274,DMSO,BRD-K90170141,...,trt,NaN,NaN,NaN,NaN,9.72,BRD-K90170141-001-01-5_9.72,CDRP,CDRP-26795,False
153383,26795,p22,26795,H-CBLD-004-4,p22,treated,BRD-K45307499-001-01-7,5.109445,DMSO,BRD-K45307499,...,trt,NaN,NaN,NaN,NaN,10.22,BRD-K45307499-001-01-7_10.22,CDRP,CDRP-26795,False
153384,26795,p23,26795,H-CBLD-004-4,p23,treated,BRD-K84521261-001-01-1,4.958284,DMSO,BRD-K84521261,...,trt,NaN,NaN,NaN,NaN,9.92,BRD-K84521261-001-01-1_9.92,CDRP,CDRP-26795,False


## Read preproessed profiles and add slope to the features

In [34]:
from scipy.signal import find_peaks

from scipy.signal import savgol_filter


def smooth_data(data, window_length=5, polyorder=3):
    return savgol_filter(data, window_length, polyorder)


def find_end_slope(data, height=None):
    peaks, _ = find_peaks(data, height=height, width=1)
    valleys, _ = find_peaks(-data, height=height, width=1)
    extermas = np.concatenate((peaks, valleys))
    if extermas.size == 0:
        return np.nan, 0

    last_peak_ind = np.max(extermas)
    slope = data[-1] - data[last_peak_ind]
    return last_peak_ind, slope


def subtract_control(group):
    batch_plate = group.name
    control_values = control_df_perplate.loc[batch_plate]
    return group - control_values


def find_end_slope2(data, height=None):
    data = smooth_data(data)
    #     min_max_indc = [np.argmax(data[3:] + 3), np.argmin(data[3:] + 3)]
    min_max_indc = [np.argmax(data), np.argmin(data)]
    last_peak_ind0 = [i for i in min_max_indc if i < len(data) - 2]
    if last_peak_ind0 == []:
        return 0, 0
    last_peak_ind = np.max(last_peak_ind0)
    last_two_points_amplitude = (data[-1] + data[-2]) / 2
    slope = (last_two_points_amplitude - data[last_peak_ind]) / (
        len(data) - last_peak_ind - 1
    )

    #     last_peak_ind = 0
    #     slope = data[-2]  # - data[-2]
    return last_peak_ind, slope

In [36]:
f_substr = "MeanFrac"
# f_substr = "RadialCV"

target_columns = [
    "Cells_RadialDistribution_"
    + f_substr
    + "_mito_tubeness_"
    + str(i)
    + "of16"
    for i in range(5, 17)
]
# for i in range(11, 17)

# dataset = "jump_orf"
# dataset = "CDRP"
# dataset = "lincs_g"
# dataset="jump_crispr"
# dataset = "jump_compound"
# dataset = "taorf"
for dataset in ["taorf", "jump_orf", "jump_crispr", "jump_compound"]:
    ##################### Read preprocessed metadata
    annot = pd.read_csv(
        mito_project_root_dir
        + "/workspace/metadata/preprocessed/annot_"
        + dataset
        + ".csv",
        dtype={"Metadata_Plate": str},
    )

    if "Metadata_Source" not in annot.columns:
        annot["Metadata_Source"] = "broad"

    # target_features_list = ds_info_dict[dataset]["target_features_list"]
    # sources = ['source_5']
    sources = annot["Metadata_Source"].unique()
    for si in sources:
        if 1:
            df_rep_level_scaled = pd.read_csv(
                save_results_dir
                + "/preprocessed_data/"
                + dataset
                + "_df_rep_level_scaled_"
                + si
                + ".csv"
            )

            control_df_perplate = (
                df_rep_level_scaled.loc[df_rep_level_scaled["ctrl_well"]]
                .groupby(["batch_plate"])[target_columns]
                .mean()
            )

            df_rep_level_scaled_meanSub = df_rep_level_scaled.groupby(
                "batch_plate"
            )[target_columns].apply(subtract_control)

            #         slope = df_rep_level_scaled[target_columns].apply(
            #             lambda x: find_end_slope2(x)[1], axis=1
            #         )

        slope = df_rep_level_scaled_meanSub.apply(
            lambda x: find_end_slope2(x)[1], axis=1
        )

        df_rep_level_scaled["slope"] = slope

        df_rep_level_scaled = df_rep_level_scaled[
            ~df_rep_level_scaled["slope"].isnull()
        ]

        pert_col = ds_info_dict[dataset]["pert_col"]
        nOfReps = df_rep_level_scaled.groupby([pert_col]).size().reset_index()
        pairWithReplicates = nOfReps.loc[nOfReps[0] != 1, :].reset_index()[
            pert_col
        ]

        df_rep_level_scaled2 = df_rep_level_scaled[
            df_rep_level_scaled[pert_col].isin(pairWithReplicates)
        ].reset_index(drop=True)

        (
            cp_features,
            cp_features_analysis_0,
        ) = extract_cpfeature_names.extract_cpfeature_names(
            df_rep_level_scaled2
        )

        cp_features_analysis = cp_features_analysis_0 + ["slope","Metadata_Count_Cells"]

        df_rep_level_scaled2 = normalize_funcs.standardize_per_catX(
            df_rep_level_scaled2, "batch_plate", ["slope","Metadata_Count_Cells"]
        ).copy()

        #     df_rep_level_scaled2 = normalize_funcs.standardize_df_columns(
        #         df_rep_level_scaled2, cp_features_analysis, "Standard"
        #     )

        DS_featureQ_rank = pd.DataFrame(
            data=df_rep_level_scaled2.groupby(pert_col)
            .std()
            .mean()[cp_features_analysis],
            index=cp_features_analysis,
            columns=["average_std_score"],
        )

        DS_featureQ_rank_sorted = (
            DS_featureQ_rank.sort_values(by="average_std_score")
            .reset_index()
            .rename(columns={"index": "features"})
        )

        print(
            dataset,
            si,
            #             DS_featureQ_rank_sorted["features"]
            #             .values.tolist()
            #             .index(target_columns[-2]),
            DS_featureQ_rank_sorted["features"]
            .values.tolist()
            .index(target_columns[-1]),
            DS_featureQ_rank_sorted["features"].values.tolist().index("slope"),
            DS_featureQ_rank_sorted["features"].values.tolist().index("Metadata_Count_Cells"),
            DS_featureQ_rank_sorted.shape[0],
        )

        

taorf broad 1594 2393 2751 3364


KeyError: "Columns not found: 'Metadata_Count_Cells'"

In [41]:
df_rep_level_scaled.columns[df_rep_level_scaled.columns.str.contains('RadialDistribution_MeanFrac.*16')]

Index(['Cells_RadialDistribution_MeanFrac_mito_tubeness_10of16',
       'Cells_RadialDistribution_MeanFrac_mito_tubeness_11of16',
       'Cells_RadialDistribution_MeanFrac_mito_tubeness_12of16',
       'Cells_RadialDistribution_MeanFrac_mito_tubeness_13of16',
       'Cells_RadialDistribution_MeanFrac_mito_tubeness_14of16',
       'Cells_RadialDistribution_MeanFrac_mito_tubeness_15of16',
       'Cells_RadialDistribution_MeanFrac_mito_tubeness_16of16',
       'Cells_RadialDistribution_MeanFrac_mito_tubeness_16of20',
       'Cells_RadialDistribution_MeanFrac_mito_tubeness_1of16',
       'Cells_RadialDistribution_MeanFrac_mito_tubeness_2of16',
       'Cells_RadialDistribution_MeanFrac_mito_tubeness_3of16',
       'Cells_RadialDistribution_MeanFrac_mito_tubeness_4of16',
       'Cells_RadialDistribution_MeanFrac_mito_tubeness_5of16',
       'Cells_RadialDistribution_MeanFrac_mito_tubeness_6of16',
       'Cells_RadialDistribution_MeanFrac_mito_tubeness_7of16',
       'Cells_RadialDistribution

In [37]:
dataset

'jump_orf'

In [ ]:
taorf broad 1594 2034 3363
taorf broad 1594 2432 2751 3364
jump_orf source_4 2897 3294 3440
jump_crispr source_13 3128 3114 3397

In [23]:
# df_rep_level_scaled_meanSub["Metadata_Sample_Dose"]

In [65]:
df_rep_level_scaled["ctrl_well"].unique()
# pert_col
# df_rep_level_scaled.columns[df_rep_level_scaled.columns.str.contains("dose")]
annot

,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_ASSAY_WELL_ROLE,Metadata_broad_sample,Metadata_mmoles_per_liter,Metadata_solvent,Metadata_pert_id,...,Metadata_pert_iname,Metadata_pert_iname2,Metadata_moa,Metadata_target,Metadata_mmoles_per_liter2,Metadata_Sample_Dose,Batch,batch_plate,ctrl_well,Metadata_Source
0,24277,a01,24277,H-BIOA-004-3,a01,treated,BRD-K18250272-003-03-7,3.022516,DMSO,BRD-K18250272,...,propoxycaine,propoxycaine,local anesthetic,NaN,6.05,BRD-K18250272-003-03-7_6.05,CDRP,CDRP-24277,False,broad
1,24277,a02,24277,H-BIOA-004-3,a02,treated,BRD-K18316707-001-01-9,5.000000,DMSO,BRD-K18316707,...,O-1918,BRD-K18316707,cannabinoid receptor antagonist,NaN,10.00,BRD-K18316707-001-01-9_10.0,CDRP,CDRP-24277,False,broad
2,24277,a03,24277,H-BIOA-004-3,a03,treated,BRD-K18438502-001-02-6,5.000000,DMSO,BRD-K18438502,...,NaN,NaN,NaN,NaN,10.00,BRD-K18438502-001-02-6_10.0,CDRP,CDRP-24277,False,broad
3,24277,a04,24277,H-BIOA-004-3,a04,treated,BRD-K18550767-001-02-8,5.000000,DMSO,BRD-K18550767,...,bergenin,bergenin,interleukin inhibitor,"IL1B, TNF",10.00,BRD-K18550767-001-02-8_10.0,CDRP,CDRP-24277,False,broad
4,24277,a05,24277,H-BIOA-004-3,a05,treated,BRD-K18574842-323-03-3,2.195487,DMSO,BRD-K18574842,...,nafcillin,nafcillin,bacterial cell wall synthesis inhibitor,"CYP1A2, CYP3A4, SLC22A6",4.39,BRD-K18574842-323-03-3_4.39,CDRP,CDRP-24277,False,broad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153381,26795,p20,26795,H-CBLD-004-4,p20,treated,BRD-K17405724-001-01-0,4.916061,DMSO,BRD-K17405724,...,NaN,NaN,NaN,NaN,9.83,BRD-K17405724-001-01-0_9.83,CDRP,CDRP-26795,False,broad
153382,26795,p21,26795,H-CBLD-004-4,p21,treated,BRD-K90170141-001-01-5,4.859274,DMSO,BRD-K90170141,...,NaN,NaN,NaN,NaN,9.72,BRD-K90170141-001-01-5_9.72,CDRP,CDRP-26795,False,broad
153383,26795,p22,26795,H-CBLD-004-4,p22,treated,BRD-K45307499-001-01-7,5.109445,DMSO,BRD-K45307499,...,NaN,NaN,NaN,NaN,10.22,BRD-K45307499-001-01-7_10.22,CDRP,CDRP-26795,False,broad
153384,26795,p23,26795,H-CBLD-004-4,p23,treated,BRD-K84521261-001-01-1,4.958284,DMSO,BRD-K84521261,...,NaN,NaN,NaN,NaN,9.92,BRD-K84521261-001-01-1_9.92,CDRP,CDRP-26795,False,broad


In [26]:
df_rep_level_scaled[df_rep_level_scaled.columns[df_rep_level_scaled.columns.str.contains('ount')]]

""
0
1
2
3
4
...
86690
86691
86692
86693


In [29]:
df_rep_level_scaled.columns[df_rep_level_scaled.columns.str.contains('Count')]

Index([], dtype='object')

In [ ]:
df_rep_level_scaled

In [43]:
df_rep_level_scaled.groupby("batch_plate").apply(subtract_control)

KeyError: 'CDRP-24277'

In [184]:
# df_rep_level_scaled2 = normalize_funcs.standardize_per_catX(
#     df_rep_level_scaled2, "batch_plate", ["slope"]
# ).copy()

df_rep_level_scaled2.groupby([pert_col]).mean()[
    target_columns + ["slope"]
].corr()["slope"]

Cells_RadialDistribution_MeanFrac_mito_tubeness_5of16     0.173152
Cells_RadialDistribution_MeanFrac_mito_tubeness_6of16     0.187656
Cells_RadialDistribution_MeanFrac_mito_tubeness_7of16     0.139238
Cells_RadialDistribution_MeanFrac_mito_tubeness_8of16     0.061741
Cells_RadialDistribution_MeanFrac_mito_tubeness_9of16    -0.026766
Cells_RadialDistribution_MeanFrac_mito_tubeness_10of16   -0.144750
Cells_RadialDistribution_MeanFrac_mito_tubeness_11of16   -0.407663
Cells_RadialDistribution_MeanFrac_mito_tubeness_12of16   -0.785510
Cells_RadialDistribution_MeanFrac_mito_tubeness_13of16   -0.383967
Cells_RadialDistribution_MeanFrac_mito_tubeness_14of16   -0.052861
Cells_RadialDistribution_MeanFrac_mito_tubeness_15of16    0.174169
Cells_RadialDistribution_MeanFrac_mito_tubeness_16of16    0.534113
slope                                                     1.000000
Name: slope, dtype: float64

In [115]:
df_rep_level_scaled = df_rep_level_scaled[
    ~df_rep_level_scaled["slope"].isnull()
]

pert_col = ds_info_dict[dataset]["pert_col"]
nOfReps = df_rep_level_scaled.groupby([pert_col]).size().reset_index()
pairWithReplicates = nOfReps.loc[nOfReps[0] != 1, :].reset_index()[pert_col]

df_rep_level_scaled2 = df_rep_level_scaled[
    df_rep_level_scaled[pert_col].isin(pairWithReplicates)
].reset_index(drop=True)

(
    cp_features,
    cp_features_analysis_0,
) = extract_cpfeature_names.extract_cpfeature_names(df_rep_level_scaled2)

cp_features_analysis = cp_features_analysis_0 + ["slope"]

df_rep_level_scaled2 = normalize_funcs.standardize_per_catX(
    df_rep_level_scaled2, "batch_plate", ["slope"]
).copy()

# df_rep_level_scaled2 = normalize_funcs.standardize_df_columns(
#     df_rep_level_scaled2, ["slope"], "Standard"
# )

DS_featureQ_rank = pd.DataFrame(
    data=df_rep_level_scaled2.groupby(pert_col)
    .std()
    .mean()[cp_features_analysis],
    index=cp_features_analysis,
    columns=["average_std_score"],
)

DS_featureQ_rank_sorted = (
    DS_featureQ_rank.sort_values(by="average_std_score")
    .reset_index()
    .rename(columns={"index": "features"})
)

print(
    dataset,
    si,
    DS_featureQ_rank_sorted["features"]
    .values.tolist()
    .index(target_columns[-1]),
    DS_featureQ_rank_sorted["features"].values.tolist().index("slope"),
)

jump_orf source_4 2897 3333


In [107]:
DS_featureQ_rank_sorted

,features,average_std_score
0,Cytoplasm_AreaShape_BoundingBoxArea,0.000936
1,Cells_AreaShape_BoundingBoxArea,0.000936
2,Cells_AreaShape_Area,0.001154
3,Cytoplasm_AreaShape_Area,0.001369
4,Nuclei_AreaShape_BoundingBoxArea,0.003192
...,...,...
3435,Cells_AreaShape_BoundingBoxMinimum_Y,12.469007
3436,Nuclei_AreaShape_BoundingBoxMaximum_Y,12.470892
3437,Nuclei_Location_Center_Y,12.474537
3438,Nuclei_AreaShape_Center_Y,12.474537


In [79]:
print(
    DS_featureQ_rank_sorted["features"]
    .values.tolist()
    .index(target_columns[-1]),
    DS_featureQ_rank_sorted["features"].values.tolist().index("slope"),
)

jump_orf source_4 2897 3364
jump_crispr source_13 3147 3177

2956 3307


## Read feature quality scores and make plots

In [169]:
Metadata_JCP2022